In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils import PkdbModel, caffeine_idx, pktype_data, filter_out, convert_unit, unstring
plt.style.use('seaborn-whitegrid')
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.data.loc[(all_complete.data[("sex","choice")] == "M"),("oral contraceptives","choice")] = "N"
complete_caffeine = all_complete.data[caffeine_idx(all_complete.data)]

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (10,17,21,22,23,46,47,50,71,72,74,75,76,77,80,81,96,98,100,103,107,111,112,114,117,129,139,140,141,143,146,148,149,152,161,163,164,165,166,186,191,202,207,213,227,233,234,241,247) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
caffeine_clearance = pktype_data(complete_caffeine,"clearance")
caffeine_clearance = filter_out(caffeine_clearance,"unit",["ml/min/1.73m^2"])
#caffeine_clearance = filter_out(caffeine_clearance,"study",["Balogh1992"])

caffeine_clearance[("weight","value")] = pd.to_numeric(caffeine_clearance[("weight","value")])
caffeine_clearance[("weight","mean")] = pd.to_numeric(caffeine_clearance[("weight","mean")])
caffeine_clearance["value_intervention"] = pd.to_numeric(caffeine_clearance["value_intervention"])
caffeine_clearance["median"] = pd.to_numeric(caffeine_clearance["median"])
caffeine_clearance["mean"] = pd.to_numeric(caffeine_clearance["mean"])
caffeine_clearance["value"] = pd.to_numeric(caffeine_clearance["value"])
caffeine_clearance["se"] = pd.to_numeric(caffeine_clearance["se"])
caffeine_clearance["sd"] = pd.to_numeric(caffeine_clearance["sd"])
caffeine_clearance["cv"] = pd.to_numeric(caffeine_clearance["cv"])

In [5]:
caffeine_clearance_rel = convert_unit(caffeine_clearance,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/caffeine_clearance[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

caffeine_clearance_abs = convert_unit(caffeine_clearance,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=caffeine_clearance[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])

caffeine_clearance = pd.concat([caffeine_clearance,caffeine_clearance_rel,caffeine_clearance_abs], ignore_index=True)

caffeine_clearance_rel = convert_unit(caffeine_clearance,
                             unit_in="l/h",
                             unit_out="l/h/kg", 
                             factor=1.0/caffeine_clearance[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=[("weight","value"),"value"])

caffeine_clearance_abs_i = convert_unit(caffeine_clearance,
                             unit_in="l/h/kg",
                             unit_out="l/h", 
                             factor=caffeine_clearance[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'],
                             subset=[("weight","value"),"value"])

caffeine_clearance = pd.concat([caffeine_clearance,caffeine_clearance_rel,caffeine_clearance_abs_i], ignore_index=True)

caffeine_clearance_rel = convert_unit(caffeine_clearance,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/caffeine_clearance[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

caffeine_clearance_abs = convert_unit(caffeine_clearance,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=caffeine_clearance[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
caffeine_clearance = pd.concat([caffeine_clearance,caffeine_clearance_rel,caffeine_clearance_abs], ignore_index=True)


caffeine_clearance_rel = convert_unit(caffeine_clearance,
                             unit_in="l/h",
                             unit_out="l/h/kg", 
                             factor=1.0/caffeine_clearance[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])

caffeine_clearance_abs = convert_unit(caffeine_clearance,
                             unit_in="l/h/kg",
                             unit_out="l/h", 
                             factor=caffeine_clearance[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


caffeine_clearance = pd.concat([caffeine_clearance,caffeine_clearance_rel,caffeine_clearance_abs], ignore_index=True)

In [6]:
caffeine_clr = PkdbModel("caffeine_clearance",destination="3-inferred")
caffeine_clr.data = caffeine_clearance
caffeine_clr.save()

## Timecourses

In [7]:
caffeine_timecourse = pktype_data(complete_caffeine,"concentration")
caffeine_timecourse = filter_out(caffeine_timecourse,"output_type",["outputs"])
caffeine_timecourse = filter_out(caffeine_timecourse,"unit",["µmol/l"])
caffeine_timecourse.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  tissue  substance_name  \
unit  unit_intervention                                                    
µg/ml mg                   101          101  101     101             101   
      mg/kg                  9            9    9       9               9   

                         pktype  interventions  individual_pk  value  \
unit  unit_intervention                                                
µg/ml mg                    101            101             84     84   
      mg/kg                   9              9              5      5   

                         group_pk           ...             \
unit  unit_intervention                     ...              
µg/ml mg                       17           ...              
      mg/kg                     4           ...              

                         (lean body mass, cv)  (lean body mass, mean)  \
unit  unit_intervention                                                 
µg/ml mg                                    0                       0   
      mg/kg                                 0                       0   

                         (lean body mass, sd)  (lean body mass, se)  \
unit  unit_intervention                                               
µg/ml mg                                    0                     0   
      mg/kg                                 0                     0   

                         (lean body mass, unit)  (metabolic ratio, count)  \
unit  unit_intervention                                                     
µg/ml mg                                      0                         0   
      mg/kg                                   0                         0   

                         (metabolic ratio, max)  (metabolic ratio, mean)  \
unit  unit_intervention                                                    
µg/ml mg                                      0                        0   
      mg/kg                                   0                        0   

                         (metabolic ratio, min)  (metabolic ratio, unit)  
unit  unit_intervention                                                   
µg/ml mg                                      0                        0  
      mg/kg                                   0                        0  

[2 rows x 250 columns]

In [8]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.data = caffeine_timecourse
caffeine_tc.to_array()
caffeine_timecourse = caffeine_tc.data

In [9]:
caffeine_timecourse_rel = convert_unit(caffeine_timecourse,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/caffeine_timecourse[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

caffeine_timecourse_abs = convert_unit(caffeine_timecourse,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=caffeine_timecourse[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])



caffeine_timecourse = pd.concat([caffeine_timecourse,caffeine_timecourse_rel,caffeine_timecourse_abs], ignore_index=True)


caffeine_timecourse_rel = convert_unit(caffeine_timecourse,
                             unit_in="µg/ml",
                             unit_out="µg/ml/kg", 
                             factor=1.0/caffeine_timecourse[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=[("weight","value"),"value"])

caffeine_timecourse = pd.concat([caffeine_timecourse,caffeine_timecourse_rel], ignore_index=True)

caffeine_timecourse_rel = convert_unit(caffeine_timecourse,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/caffeine_timecourse[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

caffeine_timecourse_abs = convert_unit(caffeine_timecourse,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=caffeine_timecourse[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
caffeine_timecourse = pd.concat([caffeine_timecourse,caffeine_timecourse_rel,caffeine_timecourse_abs], ignore_index=True)


caffeine_timecourse_rel = convert_unit(caffeine_timecourse,
                             unit_in="µg/ml",
                             unit_out="µg/ml/kg", 
                             factor=1.0/caffeine_timecourse[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


caffeine_timecourse = pd.concat([caffeine_timecourse,caffeine_timecourse_rel], ignore_index=True)

In [10]:
caffeine_timecourse.groupby(["unit","unit_intervention","time_unit"]).count()

study  output_type   pk  tissue  \
unit     unit_intervention time_unit                                    
µg/ml    mg                h            103          103  103     103   
         mg/kg             h            102          102  102     102   
µg/ml/kg mg                h             95           95   95      95   
         mg/kg             h             95           95   95      95   

                                      substance_name  pktype  interventions  \
unit     unit_intervention time_unit                                          
µg/ml    mg                h                     103     103            103   
         mg/kg             h                     102     102            102   
µg/ml/kg mg                h                      95      95             95   
         mg/kg             h                      95      95             95   

                                      individual_pk  value  group_pk  \
unit     unit_intervention time_unit                                   
µg/ml    mg                h                     84     84        19   
         mg/kg             h                     89     89        13   
µg/ml/kg mg                h                     84     84        11   
         mg/kg             h                     84     84        11   

                                               ...             \
unit     unit_intervention time_unit           ...              
µg/ml    mg                h                   ...              
         mg/kg             h                   ...              
µg/ml/kg mg                h                   ...              
         mg/kg             h                   ...              

                                      (lean body mass, cv)  \
unit     unit_intervention time_unit                         
µg/ml    mg                h                             0   
         mg/kg             h                             0   
µg/ml/kg mg                h                             0   
         mg/kg             h                             0   

                                      (lean body mass, mean)  \
unit     unit_intervention time_unit                           
µg/ml    mg                h                               0   
         mg/kg             h                               0   
µg/ml/kg mg                h                               0   
         mg/kg             h                               0   

                                      (lean body mass, sd)  \
unit     unit_intervention time_unit                         
µg/ml    mg                h                             0   
         mg/kg             h                             0   
µg/ml/kg mg                h                             0   
         mg/kg             h                             0   

                                      (lean body mass, se)  \
unit     unit_intervention time_unit                         
µg/ml    mg                h                             0   
         mg/kg             h                             0   
µg/ml/kg mg                h                             0   
         mg/kg             h                             0   

                                      (lean body mass, unit)  \
unit     unit_intervention time_unit                           
µg/ml    mg                h                               0   
         mg/kg             h                               0   
µg/ml/kg mg                h                               0   
         mg/kg             h                               0   

                                      (metabolic ratio, count)  \
unit     unit_intervention time_unit                             
µg/ml    mg                h                                 0   
         mg/kg             h                                 0   
µg/ml/kg mg                h                                 0   
         mg/kg             h                                 0   

              

In [11]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.data = caffeine_timecourse
caffeine_tc.save()